# Importer les données d'une API vers Google BigQuery, avec Google Cloud Function

Dans ce notebook, nous verrons comment écrire une cloud function dans Google Cloud Platform (en Python), afin d’importer les données de cette API 'https://restcountries.com/v2/region/Europe' vers Google Big Query
<img src='.\cloud-function\cloud-function.PNG'>

<img src='.\cloud-function\cloud-function1.PNG'>
<img src='.\cloud-function\cloud-function2.PNG'>
<img src='.\cloud-function\cloud-function3.PNG'>
<img src='.\cloud-function\cloud-function4.PNG'>


In [67]:
import json
import requests
from datetime import datetime
from dateutil import parser
from google.cloud import bigquery
from google.oauth2 import service_account


### On appelle les données de config

headers = {'Content-Type': 'application/json'}

CRED_PATH = "flash-datum-368412-1df83d779e92.json"
LOCATION = "us-central1"
CREDENTIALS = service_account.Credentials.from_service_account_file(CRED_PATH)


def get_project_id():
    with open(CRED_PATH) as f:
        data = json.load(f)
        return data['project_id']

In [70]:
PROJECT_ID = get_project_id()
CLIENT = bigquery.Client(project=PROJECT_ID, credentials=CREDENTIALS)

In [71]:
### Fonction qui va nous servir à uploader les données dans BigQUery
def save_data(json_data: json):
    try:
        dataset_name = 'country'
        table_id = "europe-country"

        dataset_id = f'{PROJECT_ID}.{dataset_name}'
        dataset = bigquery.Dataset(dataset_id)
        dataset.location = LOCATION
        table_ref = dataset.table(table_id)
        table = CLIENT.get_table(table_ref)
        errors = CLIENT.insert_rows(table, json_data)
        print(errors)
        if not errors:
            return "success"
        else:
            error = errors[0]["errors"]
            message = error[0]["message"]
            return message

    except Exception as error:
        return f"Data saving error. Details: {error}"


In [72]:
### On prépare la requête API
def extract():
    r = requests.get('https://restcountries.com/v2/region/Europe')
    data = json.loads(r.content)
    return data

In [73]:
extract()

[{'name': 'Åland Islands',
  'topLevelDomain': ['.ax'],
  'alpha2Code': 'AX',
  'alpha3Code': 'ALA',
  'callingCodes': ['358'],
  'capital': 'Mariehamn',
  'altSpellings': ['AX', 'Aaland', 'Aland', 'Ahvenanmaa'],
  'subregion': 'Northern Europe',
  'region': 'Europe',
  'population': 28875,
  'latlng': [60.116667, 19.9],
  'demonym': 'Ålandish',
  'area': 1580.0,
  'timezones': ['UTC+02:00'],
  'nativeName': 'Åland',
  'numericCode': '248',
  'flags': {'svg': 'https://flagcdn.com/ax.svg',
   'png': 'https://flagcdn.com/w320/ax.png'},
  'currencies': [{'code': 'EUR', 'name': 'Euro', 'symbol': '€'}],
  'languages': [{'iso639_1': 'sv',
    'iso639_2': 'swe',
    'name': 'Swedish',
    'nativeName': 'svenska'}],
  'translations': {'br': 'Åland',
   'pt': 'Ilhas de Aland',
   'nl': 'Ålandeilanden',
   'hr': 'Ålandski otoci',
   'fa': 'جزایر الند',
   'de': 'Åland',
   'es': 'Alandia',
   'fr': 'Åland',
   'ja': 'オーランド諸島',
   'it': 'Isole Aland',
   'hu': 'Åland-szigetek'},
  'flag': 'https:

In [74]:
def cloud_handler(request=None):
    """
        :param request:
        :return: success/error message
    """


    try:
        print(f"Start processing at: {datetime.now().isoformat()}")

### Extract data
        request_data = extract()

        json_data = []

        for i in range(len(request_data)):
            json_data.append(
                {
                    "name": request_data[i]['name'],
                    "capital": request_data[i]['capital'],
                    "population": request_data[i]['population']                }
            )
        print(json_data)
  
### store data to BigQuery
        res = save_data(json_data)
    

        if res == "success":
            return {"message": "Data saved successfully."}, 200

        print("error: " + res)
        return {"message": res}, 400

    except Exception as error_info:
        print(error_info)
        return {"message": error_info}, 400

In [75]:
cloud_handler()

Start processing at: 2023-01-05T22:08:59.949748
[{'name': 'Åland Islands', 'capital': 'Mariehamn', 'population': 28875}, {'name': 'Albania', 'capital': 'Tirana', 'population': 2837743}, {'name': 'Andorra', 'capital': 'Andorra la Vella', 'population': 77265}, {'name': 'Austria', 'capital': 'Vienna', 'population': 8917205}, {'name': 'Belarus', 'capital': 'Minsk', 'population': 9398861}, {'name': 'Belgium', 'capital': 'Brussels', 'population': 11555997}, {'name': 'Bosnia and Herzegovina', 'capital': 'Sarajevo', 'population': 3280815}, {'name': 'Bulgaria', 'capital': 'Sofia', 'population': 6927288}, {'name': 'Croatia', 'capital': 'Zagreb', 'population': 4047200}, {'name': 'Cyprus', 'capital': 'Nicosia', 'population': 1207361}, {'name': 'Czech Republic', 'capital': 'Prague', 'population': 10698896}, {'name': 'Denmark', 'capital': 'Copenhagen', 'population': 5831404}, {'name': 'Estonia', 'capital': 'Tallinn', 'population': 1331057}, {'name': 'Faroe Islands', 'capital': 'Tórshavn', 'populatio

({'message': 'Data saved successfully.'}, 200)

# Déclencher la fonction périodiquement (tous les jours)
Pour cela, il suffit d’aller dans cloud scheduler :
<img src='.\cloud-function\cloud-function7.PNG'>